<a href="https://colab.research.google.com/github/Panuwichhh/Mini-RAG-Covid-19/blob/main/Project_Mini_RAG_Covid_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install -U pandas langchain langchain-text-splitters langchain-community bs4 neo4j tiktoken langchain-neo4j langchain-huggingface sentence_transformers langchain-google-genai faiss-cpu wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
import os
import pandas as pd
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

## Environment Setup

In [ ]:
os.environ["NEO4J_URI"] = "###"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "###"
os.environ["GOOGLE_API_KEY"] = "###"

In [ ]:
csv_file_vector = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR_bcpBfom_BiTJD0KaSBCr99B-dwCa_g2b2w-FgA7S4xflekwfnJwB8h1nUOvw3tPo36VX83clqlc6/pub?output=csv'
df_faiss = pd.read_csv(csv_file_vector)

In [ ]:
texts = df_faiss['text_for_embedding'].tolist()

### Create Google Gemeni

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Vector RAG

### Create Embedding Model

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# Load the specified SentenceTransformer model
embeddings = HuggingFaceEmbeddings(model_name = "paraphrase-multilingual-MiniLM-L12-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectors = embeddings.embed_documents(["texts"])

In [ ]:
embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)
docstore = InMemoryDocstore({})
index_to_docstore_id = {}
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

print("สร้าง Vector Store สำเร็จแล้ว:", vector_store)

สร้าง Vector Store สำเร็จแล้ว: <langchain_community.vectorstores.faiss.FAISS object at 0x7e2b48f691c0>


In [ ]:
print(embedding_dim)

384


### Ingestion & Indexing

In [ ]:
from langchain_core.documents import Document

documents = []
for i, text_content in enumerate(texts):
    doc = Document(
        page_content=text_content,
        metadata={ "source": f"document_{i}" }
    )
    documents.append(doc)

vector_store.add_documents(documents)

['b623a087-f7ee-4336-8a19-40ecf6f3f56a',
 'c8516c01-1a59-46f1-a181-caba0d2c5992',
 '7efbb2a2-c5b6-4ea0-b3b1-6cc3e0139649',
 '2cc815ca-085a-4ac1-871a-e2d903d944fb',
 '7cf42a45-2e5d-415b-8398-6272c30c8da5',
 '8c117c3c-84cf-4a98-8aba-4804f06a5055',
 'e764bcac-92e5-4c07-b231-8096b793b01c',
 '1f22592d-c832-47d4-a449-90815d94d433',
 '27c1d31e-425a-48b1-ae93-3845d5fdadd0',
 '52c1e65f-61c3-408b-a8f5-11724096aaa4',
 '4eaa3c08-45c7-4570-bb83-64efc484b6f3',
 '437ede52-b03a-4620-83cd-be0e9d81a167',
 '47b62aad-7f4e-455b-a4d2-1b8c8ee0255f',
 '61873e25-a41b-4f10-a5e9-73a3388f6519',
 '95417957-74fa-488b-8c7d-b75175b0e4f1',
 'bc00e4e5-f463-4262-825a-6a6221c2af98',
 '595c210c-1e89-4272-9d40-4c9d36dfdf5d',
 '3fdc12c4-e513-47c5-9ad4-097b21f6a5ff',
 'c3f6c627-d3e8-4f30-ac23-bd21e6db88fd',
 'e906d361-ef40-4a1f-acf0-0f3f1e284185',
 '8f8f2663-dd02-438c-948b-31c6c3720690',
 '8d90076e-bde1-4766-8575-8ebff0498780',
 '37ef0fca-0aba-4a0e-944e-dd452ce42435',
 'f5ddb027-41f1-4911-88d8-ca94b9797fd3',
 '0401278c-e7c1-

### query

In [ ]:
query_test = "ข่าวโควิดในกรุงเทพมหานครมีอะไรบ้าง"
results = vector_store.similarity_search(query_test, k=3)

print("Query:", query_test)
print("\nTop 3 similar documents:")
for i, doc in enumerate(results):
    print(f"\nDocument {i+1} (Source: {doc.metadata.get('source', 'Unknown')}):\n{doc.page_content}")

Query: ข่าวโควิดในกรุงเทพมหานครมีอะไรบ้าง

Top 3 similar documents:

Document 1 (Source: document_3038):
วัดมิ่งเมืองจัดบรรยายให้ความรู้ สถานการณ์โควิด-19 และการฉีดวัคซีนป้องกันโควิด-19 ให้กับสื่อมวลชนจังหวัดน่าน รายงานโดย สวท.น่าน จังหวัดน่าน ภาคเหนือ เมื่อวันที่ 26/05/2021

Document 2 (Source: document_707):
"ศูนย์สนับสนุนการเคลื่อนย้ายผู้ติดเชื้อโควิด- 19 รายงานโดย หน่วยงานสำนักข่าว จังหวัดกรุงเทพมหานคร ภาคกลางและปริมณฑล เมื่อวันที่ 06/05/2021

Document 3 (Source: document_3313):
นครนายก ก้าวต่อไป สู้ภัยโควิด รายงานโดย สำนักงานประชาสัมพันธ์จังหวัดนครนายก จังหวัดนครราชสีมา ภาคตะวันออกเฉียงเหนือ เมื่อวันที่ 29/05/2021


### Create RAG Agent with FAISS

In [ ]:
from langchain.tools import tool
from langchain_core.documents import Document

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
  """Retrieve information to help answer a query."""
  retrieved_docs = vector_store.similarity_search(query, k=5)
  serialized = "\n\n".join([f"Source: {doc.metadata.get('source', 'Unknown')}\nContent: {doc.page_content}" for doc in retrieved_docs])
  return serialized, retrieved_docs

In [ ]:
from langchain.agents import create_agent

tools = [retrieve_context]
prompt = ("""
คุณคือผู้ช่วยสรุปข่าวอัจฉริยะที่มีทักษะการเล่าเรื่องที่ยอดเยี่ยม
หน้าที่ของคุณคือ:
1. รับคำถามและค้นหาข้อมูลจาก 'retrieve_context'
2. สรุปคำตอบโดยใช้ภาษาที่ "กระชับ ทันสมัย และเข้าใจง่าย"

เงื่อนไขสำคัญ:
- ต้องอ้างอิงข้อมูลจาก Context 100% ห้ามแต่งเติม
- หากไม่พบข้อมูล ให้ตอบกลับอย่างสุภาพว่า "ขออภัย ข้อมูลในระบบไม่เพียงพอสำหรับคำถามนี้"
""")
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt,
)

In [ ]:
query = "มาตรการป้องกันโควิดจังหวัดยะลาคืออะไร?"
for event in agent.stream(
    {"messages": [("user", query)]}, stream_mode="values"
):
  event["messages"][-1].pretty_print()

================================ Human Message =================================

มาตรการป้องกันโควิดจังหวัดยะลาคืออะไร?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (bea21fcc-156a-4108-9305-7d5f69a173d7)
 Call ID: bea21fcc-156a-4108-9305-7d5f69a173d7
  Args:
    query: มาตรการป้องกันโควิดจังหวัดยะลา
================================= Tool Message =================================
Name: retrieve_context

Source: document_1394
Content: เฝ้าระวังเข้มกลุ่มเสี่ยง โควิด เข้าพื้นที่ยะลาช่วงรายอ รายงานโดย สทท.ยะลา จังหวัดยะลา ภาคใต้ เมื่อวันที่ 12/05/2021

Source: document_651
Content: ผู้ว่าฯยะลา ลงพื้นที่ ต.บาลอ อ.รามัน ติดตามสถานการณ์โควิดในพื้นที่ หลังชาวบ้านเรียกร้องให้ภาครัฐปิดตำบล เพื่อควบคุมการแพร่กระจายของโควิด 19 รายงานโดย สวท.ยะลา จังหวัดยะลา ภาคใต้ เมื่อวันที่ 06/05/2021

Source: document_390
Content: คุมเข้ม บุคคลเข้า-ออก บ้านบาละ ยะลา  หลังล็อกดาวน์พื้นที่เสี่ยงโควิด-19 รายงานโดย สทท.ยะลา จังหวัดยะลา ภาคใต้ เมื่อว

##  Implement RAG Chain

In [ ]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
  last_query = request.state["messages"][-1].text
  retrieved_docs = vector_store.similarity_search(last_query)
  docs_content = "\n\n".join([doc.page_content for doc in retrieved_docs])
  system_message = docs_content
  return system_message
agent = create_agent(model=model, tools=[], middleware=[prompt_with_context])

In [ ]:
query = "มาตรการป้องกันโควิดจังหวัดยะลาคืออะไร?"
for step in agent.stream(
  {"messages": [("user", query)]}, stream_mode="values"
):
  step["messages"][-1].pretty_print()

================================ Human Message =================================

มาตรการป้องกันโควิดจังหวัดยะลาคืออะไร?
================================== Ai Message ==================================

จากข้อมูลที่ให้มา มาตรการป้องกันโควิด-19 ของจังหวัดยะลาที่ปรากฏมีดังนี้:

*   **การตั้งด่านชุมชน:** มีการตรวจด่านชุมชนตามรอยต่อจังหวัดเพื่อป้องกันการแพร่ระบาดของโควิด-19
*   **การลงพื้นที่ติดตามสถานการณ์:** ผู้ว่าราชการจังหวัดยะลาได้ลงพื้นที่เพื่อติดตามสถานการณ์โควิด-19 ในพื้นที่ โดยเฉพาะในตำบลบาลอ อำเภอรามัน หลังจากมีชาวบ้านเรียกร้องให้ภาครัฐปิดตำบลเพื่อควบคุมการแพร่ระบาด
*   **การล็อกดาวน์พื้นที่เสี่ยง:** มีการล็อกดาวน์พื้นที่เสี่ยงในบ้านบาละ เพื่อควบคุมการแพร่ระบาดของโควิด-19 และจำกัดการเข้า-ออกในพื้นที่ดังกล่าว

โดยรวมแล้ว มาตรการของจังหวัดยะลาเน้นไปที่การ **ควบคุมพื้นที่** การ **เฝ้าระวังและตรวจตรา** รวมถึงการ **ลงพื้นที่เพื่อประเมินสถานการณ์** เพื่อจัดการกับการแพร่ระบาดของโควิด-19


#Graph RAG

#### delete chuck from neo4j

In [ ]:
from neo4j import GraphDatabase
import os

driver = GraphDatabase.driver(
    os.environ["NEO4J_URI"],
    auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
)

with driver.session() as session:
    session.run("MATCH (c:Chunk) DETACH DELETE c")
    session.run("DROP CONSTRAINT chunk_id_unique IF EXISTS")
    session.run("DROP INDEX chunk_id_index IF EXISTS")

from langchain_neo4j import Neo4jVector
import os


#### add chuck to neo4j

In [ ]:
from langchain_neo4j import Neo4jVector
db = Neo4jVector.from_documents(
    documents=documents,
    embedding=embeddings,
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"]
)

## สร้าง AI agent ให้รู้จัก No4j

In [ ]:
from langchain.tools import tool
@tool(response_format="content_and_artifact")
def neo4j_retrieve_context(query: str):
  """Retrieve information from Neo4j to help answer a query."""
  retrieved_docs = db.similarity_search_with_score(query, k=5)
  serialized = "\n\n".join(
      (f"Source: {doc[0].metadata}\nContent: {doc[0].page_content}")
      for doc in retrieved_docs
  )
  return serialized, retrieved_docs

In [ ]:
from langchain.agents import create_agent
neo4j_tools = [neo4j_retrieve_context]
prompt = ("""
คุณคือผู้ช่วยสรุปข่าวอัจฉริยะที่มีทักษะการเล่าเรื่องที่ยอดเยี่ยม
หน้าที่ของคุณคือ:
1. รับคำถามและค้นหาข้อมูลจาก 'retrieve_context'
2. สรุปคำตอบโดยใช้ภาษาที่ "กระชับ ทันสมัย และเข้าใจง่าย"

เงื่อนไขสำคัญ:
- ต้องอ้างอิงข้อมูลจาก Context 100% ห้ามแต่งเติม
- หากไม่พบข้อมูล ให้ตอบกลับอย่างสุภาพว่า "ขออภัย ข้อมูลในระบบไม่เพียงพอสำหรับคำถามนี้"
""")
neo4j_agent = create_agent(model, neo4j_tools, system_prompt=prompt)

In [ ]:
query_neo4j = "ข่าวในจังหวัดเชียงใหม่ อยู่ในภาคไหน?"
for event in neo4j_agent.stream(
    {"messages": [{"role": "user", "content": query_neo4j}]}, stream_mode="values"
):
  event["messages"][-1].pretty_print()

================================ Human Message =================================

ข่าวในจังหวัดเชียงใหม่ อยู่ในภาคไหน?
================================== Ai Message ==================================
Tool Calls:
  neo4j_retrieve_context (c6c5fab1-8fd6-4236-b5c3-b6d98de2cd20)
 Call ID: c6c5fab1-8fd6-4236-b5c3-b6d98de2cd20
  Args:
    query: ข่าวในจังหวัดเชียงใหม่ อยู่ในภาคไหน
================================= Tool Message =================================
Name: neo4j_retrieve_context

Source: {'source': 'document_683'}
Content: ภาพรวมสถานการณ์โควิด-19 ของจังหวัดเชียงใหม่เริ่มดีขึ้น รายงานโดย สวท.เชียงใหม่ จังหวัดเชียงใหม่ ภาคเหนือ เมื่อวันที่ 06/05/2021

Source: {'source': 'document_2269'}
Content: อำเภอฝาง คุมเข้มแนวชานแดน ป้องกันโควิด  - 19 รายงานโดย สวท.ฝาง จังหวัดเชียงใหม่ ภาคเหนือ เมื่อวันที่ 19/05/2021

Source: {'source': 'document_2752'}
Content: จังหวัดเชียงใหม่ ขอความร่วมมือชาวเชียงใหม่ ช่วยกันสอดส่องดูแล แจ้งเบาะแสเพื่อหยุดการแพร่ระบาดของโรคโควิด-19 รายงานโดย สวท.เชียงใหม่ จัง

### Neo4J Implement RAG Chain

In [ ]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_neo4j_context(request: ModelRequest) -> str:
  last_query = request.state["messages"][-1].text
  retrieved_docs = db.similarity_search_with_score(last_query)
  docs_content = "\n\n".join([doc[0].page_content for doc in retrieved_docs])
  system_message = f"You are a helpful assistant. Use the following context to answer the question:\n\n{docs_content}"
  return system_message

neo4j_agent = create_agent(model=model, tools=[], middleware=[prompt_with_neo4j_context])

In [ ]:
query_neo4j = "ข่าวในจังหวัดเชียงใหม่ อยู่ในภาคไหน?"
for step in neo4j_agent.stream(
  {"messages": [("user", query_neo4j)]}, stream_mode="values"
):
  step["messages"][-1].pretty_print()

================================ Human Message =================================

ข่าวในจังหวัดเชียงใหม่ อยู่ในภาคไหน?
================================== Ai Message ==================================

ข่าวในจังหวัดเชียงใหม่ อยู่ใน **ภาคเหนือ** ครับ


# Knowledge Graph RAG

In [ ]:
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain

graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"]
)

chain = GraphCypherQAChain.from_llm(
    llm=model,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True
)

In [ ]:
chain.run("จากข่าวค่ายสุรศักดิ์มนตรีช่วยเหลือประชาชนอย่างไรบ้าง?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n:News)-[:HAPPENEDAT]->(r:Region)<-[:LOCATED_IN]-(p:Province)<-[:LOCATED_IN]-(d:Department)
WHERE d.name = "ค่ายสุรศักดิ์มนตรี"
RETURN n.title

Full Context:
[]

> Finished chain.


'จากข่าวค่ายสุรศักดิ์มนตรีช่วยเหลือประชาชนดังนี้:\n\n*   **มอบถุงยังชีพ:** ช่วยเหลือประชาชนที่ประสบภัยน้ำท่วม\n*   **ซ่อมแซมบ้านเรือน:** ช่วยเหลือบ้านเรือนที่ได้รับความเสียหายจากภัยน้ำท่วม\n*   **แจกจ่ายอาหาร:** จัดเลี้ยงอาหารให้แก่ประชาชนที่ได้รับความเดือดร้อน\n\nนอกจากนี้ ยังมีการช่วยเหลือด้านอื่นๆ ที่แสดงให้เห็นถึงความมุ่งมั่นของค่ายสุรศักดิ์มนตรีในการดูแลและช่วยเหลือประชาชนในยามยากลำบาก'